In [1]:
import pandas as pd
import numpy as np
import csv as csv
import re

In [2]:
# list paths to software and locations we will need later
idba_ud = '/shared/software/bin/idba_ud'
mash = '/shared/software/bin/mash'

In [3]:
# # # Hannah:
# df = pd.read_csv('/Users/rosekantor/data/metagenomics_mentoring/arstagnation/quality_summary.tsv', sep='\t')
# df = df.rename(columns={'sample':'sample_id'})
project = 'arstagnation'
sample_id = ['ARSTAG_AR_4_27',
             'ARSTAG_ARBF_12345_pre', 'ARSTAG_ARBF_1_post',
             'ARSTAG_ARBF_2_post', 'ARSTAG_ARBF_3_post',
             'ARSTAG_ARBF_4_post', 'ARSTAG_ARBF_5_post', 
             'ARSTAG_TAPRES_TAPRES_23', 'ARSTAG_TAPRES_TAPRES_27',
             'ARSTAG_TAPRES_TAPRES_40', 'ARSTAG_TAPRES_TAPRES_41',
             'ARSTAG_CONTROL_BFSLIDECONTROL_41', 'ARSTAG_CONTROL_MANIFB_41',
             'ARSTAG_CONTROL_MOCK1E10_111821', 'ARSTAG_CONTROL_MOCK1E8_111821']
df = pd.DataFrame()
df['sample_id'] = sample_id

# # Lauren:
# project = 'dwdsf18'
# df = pd.read_csv('/Users/rosekantor/data/metagenomics_mentoring/dwdsf18/sample_ids.txt', header=None, names=['sample_id'])
# df['combined_reads'] = f'/groups/banfield/sequences/2022/' + df.sample_id + '/raw.d/' + df.sample_id + '_trim_clean.PE.fa'

# # Jorien:
# project = 'awtp2'
# df = pd.read_csv('/Users/rosekantor/data/metagenomics_mentoring/awtp2/sample_info.csv')
# df = df.rename(columns={'ggkbase_project_name':'sample_id'})
# df['combined_reads'] = f'/groups/banfield/sequences/2022/' + df.sample_id + '/raw.d/' + df.sample_id + '_trim_clean.PE.fa'

# assign paths
assem_dir = f'/groups/banfield/projects/industrial/nelson_lab/{project}/assemblies'
reads_dir = f'/groups/banfield/projects/industrial/nelson_lab/{project}/reads/trimmed'
mash_dir = f'/groups/banfield/projects/industrial/nelson_lab/{project}/mash_analysis'

# Raw read processing

1. Run `bbmap` and `sickle` on raw reads (done for you).

2. Make symlinks to reads in our analysis folders using `ln -s` so that we can have easy access to them in the future.

3. Review the output of fastqc to check the quality of the reads.  If reads appear to be of low or questionable quality, run fastqc on the trimmed reads to check that quality of the remaining reads is higher after trimming and any concerning artifacts have been removed.

4. Count the reads before and after trimming. From within /trimmed and /raw, open a tmux session and run this command for counting reads:
    
`seqkit stat *.fastq.gz > seqkit_output.tsv`

5. Combine the sample project names table with the output table from seqkit to track the data for each sample.  How much total sequencing per sample, how many forward/reverse reads before/after trimming?

# Assembly commands

Using the sample table, generate commands for running assemblies on the cluster (via SLURM). Commands may specify whether to use the memory or high-memory nodes.

In [170]:
# we will create an empty list, iterate thru the df and generate commands, save them in the list, then turn the list into a column in our df
idba_cmd = []
for row in df.itertuples():
    # some samples may require more memory for assembly
    if 'INF' in row.sample_id or 'BAC' in row.sample_id:
        cmd = f'sbatch --partition memory --wrap "' \
              f'{idba_ud} ' \
              f'--pre_correction -r {row.combined_reads} ' \
              f'-o {assem_dir}/{row.sample_id}.idba_ud"'
    else:
        cmd = f'sbatch --wrap "' \
          f'{idba_ud} ' \
          f'--pre_correction -r {row.combined_reads} ' \
          f'-o {assem_dir}/{row.sample_id}.idba_ud"'

    idba_cmd.append(cmd)
df['idba_cmd'] = idba_cmd

# write the commands to a CSV
df['idba_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/1.assembly.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

note: alternatively you could just open a file and write the commands directly to the file as you create them (in the for loop). You could also open a jupyter notebook on biotite and run the commands directly from there (not recommended because it's harder to track what is running).

Now push this shell script up to biotite and run it: `sh assembly.sh`

# Mash commands

Compare samples to each other using minhash distances between reads files.  First, need to combine forward and reverse reads into a single stream and then create a mash sketch for each sample.  Then all samples can be pairwise compared to each other to produce a distance matrix.

In [171]:
# mash all v. all reads
mash_cmd = []
for row in df.itertuples():
    cmd = f'cat {reads_dir}/{s}_trim_clean.PE.1.fastq.gz {reads_dir}/{s}_trim_clean.PE.2.fastq.gz | {mash} sketch -m 2 -r - -I {row.sample_id} -s 10000 -o {mash_dir}/{row.sample_id}'
    mash_cmd.append(cmd)
    
df['mash_cmd'] = mash_cmd

In [172]:
# write the commands to a CSV
df['mash_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/2.mash.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

## mash dist commands
`mash paste` combines multiple sketches into a single sketch.  The first arg is output name, followed by a list of all the sketch files you want to combine

Command: `mash paste awtp2.msh *msh`

`mash dist` can sketch on the fly or take a sketch as input.  Because we are doing all-vs-all we use the same msh file as the query and reference.

Command: `mash dist awtp2.msh awtp2.msh`

# Post-assembly commands

1. Rename the scaffolds so that they have the project name in the fasta headers.
2. Check assembly quality using Itai Sharon's contig_stats.pl 
3. Filter scaffolds > 1000 bp into a new min1000.fa file for further analysis.
4. Remove extra files created by the assembler
5. Make bowtie2 indexes in their own folder

In [177]:
postassem_cmd = [] # create empty list
for row in df.itertuples():
    s = row.sample_id
    
    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_scaffold.fa'

    # include "sample_id" in headers and rename file to sample_id_scaffold.fa
    rehead = f"sed 's/scaffold/{s}/g' {assem_dir}/{s}.idba_ud/scaffold.fa > {scaffolds}"

    # get contig stats
    cstats = f'contig_stats.pl -i {scaffolds}'

    # delete extra files from assembly

    clean = f'rm {assem_dir}/{s}.idba_ud/kmer '\
            f'{assem_dir}/{s}.idba_ud/contig-* '\
            f'{assem_dir}/{s}.idba_ud/align-* '\
            f'{assem_dir}/{s}.idba_ud/graph-* '\
            f'{assem_dir}/{s}.idba_ud/local-*'

    # make directory to store bowtie2 indices in
    mdbt2 = f'mkdir {assem_dir}/{s}.idba_ud/bt2/'

    # index in prep for bowtie2 mapping to get true coverage for ggkbase
    ind = f'bowtie2-build {scaffolds} {assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa'
    
    cmd = [rehead, cstats, clean, mdbt2, ind]
    all_cmd = '; '.join(cmd) # separate all commands by semicolon (so they will be executed in order for each sample)
    postassem_cmd.append(all_cmd) # append command to list
    
df['postassem_cmd'] = postassem_cmd # add as a column to the dataframe

In [178]:
# write the commands to a CSV
df['postassem_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/3.postassem.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

# ggkbase import: mapping and gene-calling

In [63]:
shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'

self_map_cmd = []
readcounts_cmd = []
genecalls_cmd = []

for row in df.itertuples():
    
    s = row.sample_id
    bt2base = f'{assem_dir}/{s}.idba_ud/bt2/{s}_scaffold.fa'
    r1 = f'{reads_dir}/{s}_trim_clean.PE.1.fastq.gz'
    r2 = f'{reads_dir}/{s}_trim_clean.PE.2.fastq.gz'
    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    scaffolds_min1000 = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'

    # perform self-mapping and add readcounts to scaffold headers
    map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} 2> {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.log | {shrinksam} -v > {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam"'
    self_map_cmd.append(map_cmd)
    
    add_readcount = f'/groups/banfield/software/pipeline/v1.1/scripts/add_read_count.rb {assem_dir}/{s}.idba_ud/{s}_scaffold_mapped.sam {assem_dir}/{s}.idba_ud/{s}_scaffold.fa 150 > {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted'
    move_file = f'mv {assem_dir}/{s}.idba_ud/{s}_scaffold.fa.counted {assem_dir}/{s}.idba_ud/{s}_scaffold.fa'
    
    # filter for only contigs ≥1000 bp
    min1000 = f'pullseq -i {scaffolds} --min 1000 > {scaffolds_min1000}'
    
    readcounts_cmd.append(f'{add_readcount}; {move_file}; {min1000}') # append command to list
    
    # gene-calling
    call_orfs = f'prodigal -i {scaffolds_min1000} -o {scaffolds_min1000}.genes -a {scaffolds_min1000}.genes.faa -d {scaffolds_min1000}.genes.fna -m -p meta'
    call_16S = f'/groups/banfield/software/pipeline/v1.1/scripts/16s.sh {scaffolds_min1000} > {scaffolds_min1000}.16s'
    call_trnas = f'/groups/banfield/software/pipeline/v1.1/scripts/trnascan_pusher.rb -i {scaffolds_min1000} > /dev/null 2>&1'
    genecalls_cmd.append(f'{call_orfs}; {call_16S}; {call_trnas}')

df['self_map_cmd'] = self_map_cmd
df['readcounts_cmd'] = readcounts_cmd
df['genecalls_cmd'] = genecalls_cmd

In [64]:
# write the commands to a CSV
df['self_map_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/4.ggkbase_self_map.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['readcounts_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/5.ggkbase_readcounts.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['genecalls_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/6.ggkbase_genecalls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

# ggkbase: Annotation by best reciprocal blast

In [61]:
usearch = '/groups/banfield/software/pipeline/v1.1/scripts/cluster_usearch_wrev.rb'
annolookup = '/shared/software/bin/annolookup.py'

usearch_cmds = []
anno_proccess_cmds = []

for row in df.itertuples():
    s = row.sample_id
    faa = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa.genes.faa'
    
    # make usearch commands (using cluster usearch wrapper)
    search_kegg = f'sbatch --wrap "{usearch} -i {faa} -k -d kegg --nocluster"'
    search_uni = f'sbatch --wrap "{usearch} -i {faa} -k -d uni --nocluster"'
    search_uniprot = f'sbatch --wrap "{usearch} -i {faa} -k -d uniprot --nocluster"'
    
    usearch_cmds.append(f'{search_kegg}; {search_uni}; {search_uniprot}')

    # gzip files
    gzip_b6 = f'gzip {faa}*.b6'
    
    # annolookup
    anno_kegg = f'{annolookup} {faa}-vs-kegg.b6.gz kegg > {faa}-vs-kegg.b6+'
    anno_uni = f'{annolookup} {faa}-vs-uni.b6.gz uniref > {faa}-vs-uni.b6+'
    anno_uniprot = f'{annolookup} {faa}-vs-uniprot.b6.gz uniprot > {faa}-vs-uniprot.b6+'
    
    anno_proccess_cmds.append(f'{gzip_b6}; {anno_kegg}; {anno_uni}; {anno_uniprot}')

df['usearch_cmds'] = usearch_cmds
df['anno_proccess_cmds'] = anno_proccess_cmds    

In [62]:
# write the commands to a CSV
df['usearch_cmds'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/7.ggkbase_usearch.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['anno_proccess_cmds'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/8.ggkbase_anno_process.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

The files have now been generated for ggkbase import.

# Anvi'o process contigs

See https://merenlab.org/2016/06/18/importing-taxonomy/ and https://merenlab.org/2016/06/22/anvio-tutorial-v2/

In [60]:
awk = "awk '{print $1}'" # using this to remove additional fields from fasta headers for contigs, ignore if you don't have this issue
anvio = 'conda activate anvio-7.1' # note that this didn't actually work, needed to use conda run but we used system anvio v7

kaiju_path = '/groups/banfield/projects/industrial/nelson_lab/kaiju/bin'
kaiju_nodes = '/shared/db/kaiju/nr_euk/r2021-02-24/nodes.dmp'
kaiju_names = '/shared/db/kaiju/nr_euk/r2021-02-24/names.dmp'

kaiju_nr = '/shared/db/kaiju/nr_euk/r2021-02-24/kaiju_db_nr_euk.fmi'
kaiju = f'{kaiju_path}/kaiju -t {kaiju_nodes} -f {kaiju_nr}'
kaiju_addtaxnames = f'{kaiju_path}/kaiju-addTaxonNames -t {kaiju_nodes} -n {kaiju_names} -r superkingdom,phylum,order,class,family,genus,species'

fix_scaffolds_cmd = []
analyze_contigs_cmd = []
addTaxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    # define file names
    scaffolds_badheaders = f'{assem_dir}/{s}.idba_ud/{s}_scaffold_min1000.fa'
    scaffolds_min1000 = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    gene_calls = f'{assem_dir}/{s}.idba_ud/{s}_gene_calls.fa'
    kaiju_out = f'{assem_dir}/{s}.idba_ud/{s}_kaiju.out'
    kaiju_processed = f'{assem_dir}/{s}.idba_ud/{s}_genes_kaiju.txt'
    
    # fix headers
    cmd = f'{awk} {scaffolds_badheaders} > {scaffolds_min1000}'
    fix_scaffolds_cmd.append(cmd)
    
    # write commands for contigs analysis on cluster
    make_cdb = f'anvi-gen-contigs-database -f {scaffolds_min1000} -o {contigsDB} -n {s} -T 48'
    get_genes = f'anvi-get-sequences-for-gene-calls -c {contigsDB} -o {gene_calls}'
    anvhmms = f'anvi-run-hmms -c {contigsDB} -T 48'
    anvscg = f'anvi-run-scg-taxonomy -c {contigsDB} -T 48' # this didn't actually run because it wasn't set up yet!
    #anvcogs = f'/shared/software/bin/anvi-run-ncbi-cogs -c {contigsDB} -T 48'
    run_kaiju = f'{kaiju} -i {gene_calls} -v -z 48 > {kaiju_out}' 
    cmd = f'sbatch --wrap "{anvio}; {make_cdb}; {get_genes}; {anvhmms}; {anvscg}; {run_kaiju}"'
    analyze_contigs_cmd.append(cmd)
    
    ## write commands for local jobs after cluster jobs
    process_kaiju = f'{kaiju_addtaxnames} -i {kaiju_out} -o {kaiju_processed}' 
    import_kaiju = f'anvi-import-taxonomy-for-genes -i {kaiju_processed} -c {contigsDB} -p kaiju --just-do-it'
    cmd = f'{process_kaiju}; {import_kaiju}'
    addTaxonomy_cmd.append(cmd)

# add to dataframe
df['fix_scaffolds_cmd'] = fix_scaffolds_cmd
df['analyze_contigs_cmd'] = analyze_contigs_cmd
df['addTaxonomy_cmd'] = addTaxonomy_cmd

# save files
df['fix_scaffolds_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_0-fix_scaffold_names.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['analyze_contigs_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_1-analyze_contigs.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
df['addTaxonomy_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_2-addTaxonomy.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

you must run `conda activate anvio-7.1` in terminal before running the addTaxonomy commands

In [153]:
# run-scg failed because set-up-scg hadn't been run so rerunning

scg_taxonomy_cmd = []

for row in df.itertuples():
    s = row.sample_id
    
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'

    anvscg = f'anvi-run-scg-taxonomy -c {contigsDB} -T 16' # this didn't actually run because it wasn't set up yet!
    scg_taxonomy_cmd.append(anvscg)

df['scg_taxonomy_cmd'] = scg_taxonomy_cmd

# save files
df['scg_taxonomy_cmd'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_scg_taxonomy_cmd.sh', index=False, quoting=csv.QUOTE_NONE, header=False, sep='\t')

# Cross-mapping reads and anvio profile

Build bt2 directories for our min1000 scaffolds to match the Anvio contigs db scaffolds.

In [75]:
with open(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_3-bt2build.sh', 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
        bt2ind = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
        # index in prep for bowtie2 mapping to get coverage on min1000 scaffolds
        f.write(f'bowtie2-build {scaffolds} {bt2ind}\n')

In [141]:
shrinksam = '/shared/software/bin/shrinksam'
bt2 = '/shared/software/bin/bowtie2'
anvio = 'conda run -n anvio-7.1'

xmapping_df = []
for srow in df.itertuples():  
    s = srow.sample_id

    scaffolds = f'{assem_dir}/{s}.idba_ud/{s}_min1000.fa'
    bt2base = f'{assem_dir}/{s}.idba_ud/bt2/{s}_min1000.fa'
    contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
    
    # map against other samples and controls
    for rrow in df.itertuples():
        r = rrow.sample_id
        r_fixed = re.sub('[\.-]', '_', r) # if your sample name has dashes or periods in it, Anvio won't like it.
        
        r1 = f'{reads_dir}/{r}_trim_clean.PE.1.fastq.gz'
        r2 = f'{reads_dir}/{r}_trim_clean.PE.2.fastq.gz'        
        raw_bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw.bam'
        filtered_bam_raw = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}-raw-filt.bam'
        bam = f'{assem_dir}/{s}.idba_ud/{s}-vs-{r}.bam'
        
        if 'CTRL' in r or 'CONTROL' in r:
            
            # filter mapping - for decontamination, we want to use filtered mappings
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {raw_bam}; ' \
                      f'reformat.sh in={raw_bam} out={filtered_bam_raw} editfilter=2 threads=48; rm {raw_bam}"'
        
        else:
            map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        # map_cmd = f'sbatch --wrap "{bt2} -p 48 -x {bt2base} -1 {r1} -2 {r2} | {shrinksam} -v | sambam > {filtered_bam_raw}"'
        
        # process mapping
        initbam = f'anvi-init-bam {filtered_bam_raw} -o {bam}; rm {filtered_bam_raw}'
        
        # anvi-profile
        profile_out = f'{assem_dir}/{s}.idba_ud/anvio_data/{r}_profile' # check this
        anvip_cmd = f'sbatch --wrap "{anvio} anvi-profile --min-contig-length 1000 --skip-SNV-profiling -T 48 -i {bam} -c {contigsDB} -o {profile_out} -S {r_fixed}"'
        
        ## append all commands to table
        xmapping_df.append([s, r, map_cmd, initbam, anvip_cmd])
xmapping_df = pd.DataFrame.from_records(xmapping_df, columns=['assembly', 'reads', 'map', 'initbam', 'anvi-profile'])

In [142]:
xmapping_df.to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/crossmapping.csv', index=False)

In [113]:
# example of filtering the xmapping_df to get just the rows you want
pos = xmapping_df[(xmapping_df.assembly == 'AWTP2_CTRL_POS-POWER9_EXTRACTION_0') 
                  & ((xmapping_df.reads.str.contains('POS')) | (xmapping_df.reads.str.contains('BLANK')))]

pos['map'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_xmap_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
pos['initbam'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_initbam_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)
pos['anvi-profile'].to_csv(f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_anvip_pos-controls.sh', index=False, quoting=csv.QUOTE_NONE, header=False)

Based on MASH distances between samples, decide which samples to cross-map against one another. A rule of thumb is that cross-mapping beyond 12 samples likely will not improve your bins.

# Binning

Note: you may want to tell concoct how many bins to create based on the counts of the SCGs in your samples.

To do this, you could add a column to your df with that information and then uncomment the lines below (and include in the concoct command)

One of the Anvi'o tutorials suggests it's better to underestimate the number of genomes since it is easier to split contaminated bins with anvi-refine than to combine fragmented bins.

In [13]:
outfile = f'/Users/rosekantor/data/metagenomics_mentoring/{project}/anvio_concoct.sh'
with open(outfile, 'w') as f:
    for row in df.itertuples():
        s = row.sample_id
        # c = row.scg_count
        profileDB = f'{assem_dir}/{s}.idba_ud/anvio_data/merged/PROFILE.db'
        contigsDB = f'{assem_dir}/{s}.idba_ud/{s}_contigs.db'
        out = f'{assem_dir}/{s}.idba_ud/anvio_data'

        get_cococt_input = f'anvi-export-splits-and-coverages -p {profileDB} -c {contigsDB} -o {out} -O {s} --splits-mode'
        run_concoct = f'concoct --coverage_file {out}/{s}-COVs.txt --composition_file {out}/{s}-SPLITS.fa -b {out}/{s}_concoct -r 150 -t 10' #-c {c}
        csv_to_tsv = f'sed "s/,/\\t/g" {out}/{s}_concoct_clustering_gt1000.csv > {out}/{s}_concoct_clustering_gt1000.tsv'
        import_collection = f'anvi-import-collection {s}_concoct_clustering_gt1000.tsv -p {profileDB} -c {contigsDB} -C concoct'

        f.write(f'{get_cococt_input}; {run_concoct}; {csv_to_tsv}; {import_collection}\n')

# using regular expressions to parse file

In [66]:
import re
pat = r'Total number of sequences: (\d+)'

In [65]:
file = '/Users/rosekantor/data/metagenomics_mentoring/ARSTAG/ARSTAG_AR_1_23_scaffold.fa.summary.txt'

with open(file, 'r') as f:
    for line in f:
        if 'Total number of sequences' in line:
            scaffold_count = re.match(pat, line).groups(0)

Total number of sequences: 78893

('78893',)


# Next steps

Post-assembly
- assess assembly quality, consider/test coassemblies

Prep for ggkbase:
- map to get true coverage for ggkbase
- gene calls for ggkbase
- annotation for ggkbase
- import into ggkbase projects

Prep for anvio:
- anvio gen-contigs-db and gen-profile-db
- kaiju for taxonomy and import into anvio
- import annotations (from ggkbase) into anvio

Binning:
- cross-mapping for binning
- auto-bin with DasTool
- refine bins with ggkbase or anvio or both
- evaluate bins with checkM2
- dereplicate bins across samples with dRep
- decontaminate assemblies using negative controls (also decontaminate reads)

Additional questions:
- annotate with special HMMs (e.g. ARGs, Metabolic)
- investigate metabolisms of key organisms of interest
- Virsorter2 to look for phage
- look for pathogens
- phylogenetic trees for key organisms of interest
- assess growth via iRep